# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Big Data** </center>
---
### <center> **Spring 2025** </center>
---
### <center> **Streamer Structrure for final Proyect** </center>
### <center> **Par de Foraneos** </center>


In [1]:
import findspark
findspark.init()
import ta
from foraneos.utils_proyectofinal_copy import resample_and_aggregate
from foraneos.utils_proyectofinal_copy import SparkUtils as SpU

## Spark Session creation


In [2]:
from pyspark.sql import SparkSession

SPARK_SERVER = {'Konrad': '672e28abb623',
                'Aaron' : 'a5ab6bdab4b3'}
KAFKA_SERVER = {'Konrad': 'dee5c9cc3710:9093',
                'Aaron' : '69b1b3611d90:9093'}
current_user = 'Konrad'

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Kafka") \
    .master("spark://{}:7077".format(SPARK_SERVER[current_user])) \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()
    
sc = spark.sparkContext

spark.conf.set("spark.sql.shuffle.partitions", "5")

:: loading settings :: url = jar:file:/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-32bd4ffb-928b-40d1-813d-0384ee9a6b09;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;3.5.4 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;3.5.4 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.scala-lang.modules#scala-parallel-collections_2.13;1.0.4 in central
	found org.apache.commons#commons-pool2;2.11.1 in centr

## Kafka Stream creation

In [3]:
streamer_lines = []

for i in range(4):
    streamer_lines.append( spark \
                            .readStream \
                            .format("kafka") \
                            .option("kafka.bootstrap.servers", "{}".format(KAFKA_SERVER[current_user])) \
                            .option("subscribe", f"stock_topic{i}") \
                            .load()
    )


In [ ]:
result_schema = SpU.generate_schema([("timestamp", "timestamp" ), 
                                              ("open", "float" ), 
                                              ("high", "float" ), 
                                              ("low", "float"),
                                              ("close", "float" ) ,
                                              ("williams_r", "float" ),  
                                              ("rsi", "float"), 
                                              ("ultimate_osc", "float"), 
                                              ("ema", "float"), 
                                              ("close_lag_1", "float" ), 
                                              ("close_lag_2", "float" ), 
                                              ("close_lag_3", "float" ), 
                                              ("close_lag_4", "float" ), 
                                              ("close_lag_5", "float" ),                                                                                 
                                              ])





## Transform binary data into string

In [5]:
from pyspark.sql.functions import col, split
from pyspark.sql.types import DoubleType, TimestampType
from pyspark.sql.functions import pandas_udf

streamer_df = []

for i in range(4):
       
           
    df = streamer_lines[i].withColumn("value_str", col("value").cast("string"))
    df = df.withColumn("split", split(col("value_str"), ","))
    df = df.withColumn("timestamp", col("split").getItem(0).cast(TimestampType())) \
           .withColumn("company", col("split").getItem(1)) \
           .withColumn("close", col("split").getItem(2).cast(DoubleType())) \
          .select("timestamp", "company","close")
          
    df.printSchema()
    custom_resampler = resample_and_aggregate(new_window=5)
    resampled_df = df.mapInPandas(custom_resampler, schema=result_schema)

    streamer_df.append(resampled_df)
    resampled_df.printSchema()



root
 |-- timestamp: timestamp (nullable = true)
 |-- company: string (nullable = true)
 |-- close: double (nullable = true)

root
 |-- timestamp: timestamp (nullable = true)
 |-- open: float (nullable = true)
 |-- high: float (nullable = true)
 |-- low: float (nullable = true)
 |-- close: float (nullable = true)
 |-- williams_r: float (nullable = true)
 |-- rsi: float (nullable = true)
 |-- ultimate_osc: float (nullable = true)
 |-- close_lag_1: float (nullable = true)
 |-- close_lag_2: float (nullable = true)
 |-- close_lag_3: float (nullable = true)
 |-- close_lag_4: float (nullable = true)
 |-- close_lag_5: float (nullable = true)

root
 |-- timestamp: timestamp (nullable = true)
 |-- company: string (nullable = true)
 |-- close: double (nullable = true)

root
 |-- timestamp: timestamp (nullable = true)
 |-- open: float (nullable = true)
 |-- high: float (nullable = true)
 |-- low: float (nullable = true)
 |-- close: float (nullable = true)
 |-- williams_r: float (nullable = true)


### Watermarking to handle late arrival events

### Sink configuration

In [ ]:
query = []

# for i in range(4):
#     query.append(
#         streamer_df[i] \
#         .writeStream \
#         .outputMode("append") \
#         .trigger(processingTime='120 seconds') \
#         .format("parquet") \
#         .option("path", f"/home/jovyan/notebooks/data/final_project_ParDeForaneos/output{i}/")
#         .option("checkpointLocation", f"/home/jovyan/notebooks/data/final_project_ParDeForaneos/checkpoints/stock_topic{i}") \
#         .start()
#     )

for i in range(4):
    query.append(
        streamer_df[i] \
        .writeStream \
        .outputMode("append") \
        .trigger(processingTime='120 seconds') \
        .format("console") \
        .start()
    )

    #query[i].awaitTermination(100)


25/05/13 01:30:49 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-f3538344-09fa-47ca-8ec5-c2be9fc9756b. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/05/13 01:30:49 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/13 01:30:49 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-a8e313ab-bd72-4596-ae0e-98db97aeff48. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/05/13 01:30:49 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not support

25/05/13 01:30:49 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/05/13 01:30:49 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/05/13 01:30:49 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/05/13 01:30:49 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
-------------------------------------------
Batch: 0
-------------------------------------------
-------------------------------------------
Batch: 0
-------------------------------------------
-------------------------------------------
Batch: 0
-------------------------------------------
+---------+----+----+---+-----+----------+---+------------+-----------+-----------+-----------+-----------+-----------+
|timestamp|open|high|low|close|williams_r|rsi|ultimate_osc|close_lag_1|close_lag_2|close_lag_3|close_lag_4|close_lag_5|
+---------+----+----+---+-----+----------+---+------------+-----------+-----------+-----------+-----------+-----------+
+---------+----+----+---+-----+----------+---+------------+-----------+-----------+-----------+-----------+-----------+

+---------+----+----+---+-----+----------+---+------------+-----------+-----------+-----------+-----------+-----------+
|timestamp|

25/05/13 01:32:06 WARN TaskSetManager: Lost task 0.0 in stage 4.0 (TID 4) (172.18.0.4 executor 0): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py", line 470, in dump_stream
    return ArrowStreamSerializer.dump_stream(self, init_stream_yield_batches(), stream)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py", line 100, in dump_stream
    for batch in iterator:
  File "/opt/conda/spark-3.5.4-bin-hadoop3-scala2.1

In [7]:
for i in range(4):
    query[i].stop()

In [ ]:
df = spark.read.parquet("/home/jovyan/notebooks/data/final_project_ParDeForaneos/output0/part-00000-cc734dfc-75e1-436c-a412-6da58b98bef5-c000.snappy.parquet")

In [ ]:
df.show()


In [8]:
sc.stop()

25/05/13 01:30:20 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /tmp/temporary-56d8ca74-e634-421b-8bc3-cd1920bd23d6. Falling back to Java IO way
java.io.IOException: Failed to delete: /tmp/temporary-56d8ca74-e634-421b-8bc3-cd1920bd23d6
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:174)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:109)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:90)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively(SparkFileUtils.scala:121)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively$(SparkFileUtils.scala:120)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils.scala:1126)
	at org.apache.spark.util.ShutdownHookManager$.$anonfun$new$4(ShutdownHookManager.scala:65)
	at org.apache.spark.util.ShutdownHookManager$.$anonfun$new$4$adapted(ShutdownHookManager.scala:62)
	at scala.collection.ArrayOps$.foreac